# Benchmark

In [ ]:
!python -m pip install pandas plotly pyyaml

In [ ]:
import pandas as pd, numpy as np
import os, glob, datetime, time
import plotly as plotly
import plotly.express as px
import hjson
import yaml

In [ ]:
# Compile hardware for Questa (vsim)
!questa-2022.3 make bin/snitch_cluster.vsim

In [ ]:
# Compile software
!make DEBUG=ON sw

In [ ]:
# Post process traces
!make -j traces
!make logs/perf.csv

In [ ]:
# Read profile data
perf = pd.read_csv('logs/perf.csv', index_col=0)
perf.filter(regex=("1_."))

In [ ]:
fig = px.scatter(perf, y=['1_total_ipc', '1_fpss_occupancy', '1_fpss_fpu_occupancy', '1_snitch_occupancy'])
fig.update_layout(yaxis_range=[0,1])

In [ ]:
def run(cmd, env=None, dryrun=False):
    if dryrun:
        print(cmd)
    else:
        p = subprocess.Popen(cmd, env=env, shell=True)
        retcode = p.wait()
        if retcode != 0:
            sys.exit(retcode)

In [ ]:
!f'make CFG_OVERRIDE={cfg_file} rtl'

# Benchmark Configuration

In [ ]:
# Load top-level benchmark config
bench_config_name = "bench/bench.yaml"
with open(bench_config_name) as f:
    bench_config = yaml.load(f, Loader=yaml.Loader)
bench_config

In [ ]:
# Expand python strings
for app in bench_config['sw']:
    if type(app['sweep']['n']) == str:
        app['sweep']['n'] = eval(app['sweep']['n'])
bench_config

In [ ]:
# flatten into a table
sw = pd.json_normalize(bench_config['sw'])
for col in sw.columns.tolist():
    if col.startswith('sweep'):
        sw = sw.explode(col)
configs = pd.DataFrame({'hw':bench_config['hw']}).merge(sw, how='cross')
configs

In [ ]:
with open("bench/test.yaml", 'w') as f:
    yaml.dump(bench_config, f)